In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install datasets transformers

In [ ]:
import pandas as pd
import numpy as np
import torch
import random
from transformers import (
    DistilBertTokenizer,
    DistilBertForSequenceClassification,
    TrainingArguments
)
from datasets import Dataset
from sklearn.preprocessing import StandardScaler
import nltk
import re
import warnings
warnings.filterwarnings('ignore')

# Download NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Set random seeds
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
#Load data
train_df = pd.read_csv('/content/drive/MyDrive/Data/train.csv', encoding='latin-1')
test_df = pd.read_csv('/content/drive/MyDrive/Data/test.csv',encoding='latin-1')

print("Training Dataset Shape:", train_df.shape)
print("Testing Dataset Shape:", test_df.shape)
print("\nTraining Label Distribution:")
print(train_df['label'].value_counts())
print("\nTesting Label Distribution:")
print(test_df['label'].value_counts())

Training Dataset Shape: (2838, 116)
Testing Dataset Shape: (715, 116)

Training Label Distribution:
label
1    1488
0    1350
Name: count, dtype: int64

Testing Label Distribution:
label
1    369
0    346
Name: count, dtype: int64


In [ ]:
def clean_text(text):
    """Clean and preprocess text"""
    text = str(text).lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    text = re.sub(r'[^a-zA-Z\s!?.]', '', text)
    text = ' '.join(text.split())
    return text.strip()

def get_important_features(df):
    """Extract important LIWC and sentiment features"""
    important_features = [
        'lex_liwc_Analytic', 'lex_liwc_Clout', 'lex_liwc_Authentic',
        'lex_liwc_Tone', 'lex_liwc_posemo', 'lex_liwc_negemo',
        'lex_liwc_anx', 'lex_liwc_anger', 'lex_liwc_sad',
        'lex_liwc_social', 'sentiment'
    ]
    return df[important_features]


In [ ]:
# Clean text
train_df['cleaned_text'] = train_df['text'].apply(clean_text)
test_df['cleaned_text'] = test_df['text'].apply(clean_text)

# Get and scale LIWC features
scaler = StandardScaler()
train_liwc = scaler.fit_transform(get_important_features(train_df))
test_liwc = scaler.transform(get_important_features(test_df))

# Convert to tensors
train_liwc_tensor = torch.FloatTensor(train_liwc)
test_liwc_tensor = torch.FloatTensor(test_liwc)

In [ ]:
# Initialize tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Create datasets
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Tokenize function
def tokenize_function(examples):
    return tokenizer(
        examples['cleaned_text'],
        padding='max_length',
        truncation=True,
        max_length=128,
        return_tensors='pt'
    )

# Tokenize datasets

train_encoded = train_dataset.map(tokenize_function, batched=False)
test_encoded = test_dataset.map(tokenize_function, batched=False)

Map:   0%|          | 0/2838 [00:00<?, ? examples/s]

Map:   0%|          | 0/715 [00:00<?, ? examples/s]

In [ ]:
# Modified model initialization with correct configuration
from transformers import DistilBertConfig

# Create config with modified dropout
config = DistilBertConfig.from_pretrained(
    'distilbert-base-uncased',
    num_labels=2,
    output_hidden_states=True,
    dropout=0.5,  # This is the correct parameter name
    attention_dropout=0.5  # This is the correct parameter name
)

# Initialize model with config
model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    config=config
).to(device)

# Modified LIWC layer with stronger regularization
liwc_layer = torch.nn.Sequential(
    torch.nn.Linear(11, 32),
    torch.nn.BatchNorm1d(32),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.5),
    torch.nn.LayerNorm(32)
).to(device)

# Modified classifier with stronger regularization
classifier = torch.nn.Sequential(
    torch.nn.Linear(768 + 32, 256),
    torch.nn.BatchNorm1d(256),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.5),
    torch.nn.Linear(256, 64),
    torch.nn.BatchNorm1d(64),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.5),
    torch.nn.Linear(64, 2)
).to(device)

# Add L2 regularization to optimizer
optimizer = torch.optim.AdamW(
    [
        {'params': model.parameters(), 'weight_decay': 0.1},
        {'params': liwc_layer.parameters(), 'weight_decay': 0.1},
        {'params': classifier.parameters(), 'weight_decay': 0.1}
    ],
    lr=1e-5
)

# Add learning rate scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',
    factor=0.5,
    patience=2,
    verbose=True
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Modified training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,            # Increased to allow for slower learning
    per_device_train_batch_size=32, # Increased for better stability
    per_device_eval_batch_size=32,
    warmup_ratio=0.1,
    weight_decay=0.1,              # Increased from 0.01
    learning_rate=1e-5,            # Decreased from 2e-5
    gradient_accumulation_steps=2,
    fp16=True,
    eval_steps=100,                # More frequent evaluation
    logging_steps=100,
    save_steps=100,
    # The fix is here. This makes sure the evaluation happens every 100 steps as well
    evaluation_strategy="steps",
    load_best_model_at_end=True,
    metric_for_best_model = "eval_loss" # what metric to use for best model
)

In [ ]:
# Create DataLoader and modify data preparation
from torch.utils.data import DataLoader, TensorDataset

def prepare_data_loaders(train_encoded, test_encoded, train_liwc_tensor, test_liwc_tensor, batch_size=16):
    # Convert training data
    train_input_ids = torch.tensor([item for item in train_encoded['input_ids']])
    train_attention_mask = torch.tensor([item for item in train_encoded['attention_mask']])
    train_labels = torch.tensor(train_encoded['label'])

    # Create training TensorDataset
    train_dataset = TensorDataset(
        train_input_ids,
        train_attention_mask,
        train_labels,
        train_liwc_tensor
    )

    # Convert test data
    test_input_ids = torch.tensor([item for item in test_encoded['input_ids']])
    test_attention_mask = torch.tensor([item for item in test_encoded['attention_mask']])
    test_labels = torch.tensor(test_encoded['label'])

    # Create test TensorDataset
    test_dataset = TensorDataset(
        test_input_ids,
        test_attention_mask,
        test_labels,
        test_liwc_tensor
    )

    # Create DataLoaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size
    )

    return train_loader, test_loader

# Also, let's modify the tokenization function to return tensors directly
def tokenize_function(examples):
    return tokenizer(
        examples['cleaned_text'],
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors=None  # Changed this to return lists instead of tensors
    )

# Re-run tokenization
train_encoded = train_dataset.map(tokenize_function, batched=True)
test_encoded = test_dataset.map(tokenize_function, batched=True)

# Create data loaders
train_loader, test_loader = prepare_data_loaders(
    train_encoded,
    test_encoded,
    train_liwc_tensor,
    test_liwc_tensor
)


Map:   0%|          | 0/2838 [00:00<?, ? examples/s]

Map:   0%|          | 0/715 [00:00<?, ? examples/s]

In [ ]:
# Training functions
def train_epoch(model, train_loader, optimizer, device):
    model.train()
    total_loss = 0

    for batch_idx, (input_ids, attention_mask, labels, liwc_features) in enumerate(train_loader):
        # Move data to device
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)
        liwc_features = liwc_features.to(device)

        # Forward pass through BERT
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        # Get the hidden states from the output
        bert_features = outputs.hidden_states[-1][:, 0, :]  # Get CLS token embedding

        # Process LIWC features
        liwc_output = liwc_layer(liwc_features)

        # Combine features
        combined_features = torch.cat((bert_features, liwc_output), dim=1)
        logits = classifier(combined_features)

        # Calculate loss
        loss_fct = torch.nn.CrossEntropyLoss()
        loss = loss_fct(logits, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        total_loss += loss.item()

        if batch_idx % 10 == 0:
            print(f'Batch {batch_idx}, Loss: {loss.item():.4f}')

    return total_loss / len(train_loader)

def evaluate(model, test_loader, device):
    model.eval()
    total_loss = 0
    predictions = []
    true_labels = []

    with torch.no_grad():
        for input_ids, attention_mask, labels, liwc_features in test_loader:
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)
            liwc_features = liwc_features.to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )

            bert_features = outputs.hidden_states[-1][:, 0, :]
            liwc_output = liwc_layer(liwc_features)
            combined_features = torch.cat((bert_features, liwc_output), dim=1)
            logits = classifier(combined_features)

            loss_fct = torch.nn.CrossEntropyLoss()
            loss = loss_fct(logits, labels)

            total_loss += loss.item()
            predictions.extend(torch.argmax(logits, dim=1).cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    return total_loss / len(test_loader), predictions, true_labels


In [ ]:
# Initialize model with output_hidden_states=True
model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=2,
    output_hidden_states=True  # Add this parameter
).to(device)

# Rest of the training code remains the same
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
optimizer = torch.optim.AdamW(
    list(model.parameters()) +
    list(liwc_layer.parameters()) +
    list(classifier.parameters()),
    lr=2e-5
)

num_epochs = 5
best_val_loss = float('inf')
patience = 3
no_improve_count = 0

for epoch in range(num_epochs):
    print(f'\nEpoch {epoch+1}/{num_epochs}')

    # Train
    train_loss = train_epoch(model, train_loader, optimizer, device)

    # Evaluate
    val_loss, predictions, true_labels = evaluate(model, test_loader, device)

    print(f'Training Loss: {train_loss:.4f}')
    print(f'Validation Loss: {val_loss:.4f}')

    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        no_improve_count = 0
        # Save best model
        torch.save({
            'model_state_dict': model.state_dict(),
            'liwc_layer_state_dict': liwc_layer.state_dict(),
            'classifier_state_dict': classifier.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
        }, 'best_model.pt')
    else:
        no_improve_count += 1
        if no_improve_count >= patience:
            print('Early stopping triggered')
            break

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Epoch 1/5
Batch 0, Loss: 0.8563
Batch 10, Loss: 0.8065
Batch 20, Loss: 0.9147
Batch 30, Loss: 0.6521
Batch 40, Loss: 0.5790
Batch 50, Loss: 0.6341
Batch 60, Loss: 0.6415
Batch 70, Loss: 0.4895
Batch 80, Loss: 0.5518
Batch 90, Loss: 0.5641
Batch 100, Loss: 0.3475
Batch 110, Loss: 0.3664
Batch 120, Loss: 0.5451
Batch 130, Loss: 0.4194
Batch 140, Loss: 0.5050
Batch 150, Loss: 0.4998
Batch 160, Loss: 0.5155
Batch 170, Loss: 0.2392
Training Loss: 0.6176
Validation Loss: 0.5523

Epoch 2/5
Batch 0, Loss: 0.5260
Batch 10, Loss: 0.5195
Batch 20, Loss: 0.4529
Batch 30, Loss: 0.7997
Batch 40, Loss: 0.4849
Batch 50, Loss: 0.4029
Batch 60, Loss: 0.3642
Batch 70, Loss: 0.4761
Batch 80, Loss: 0.3905
Batch 90, Loss: 0.3513
Batch 100, Loss: 0.5187
Batch 110, Loss: 0.4709
Batch 120, Loss: 0.5669
Batch 130, Loss: 0.4038
Batch 140, Loss: 0.7363
Batch 150, Loss: 0.4265
Batch 160, Loss: 0.4231
Batch 170, Loss: 0.3456
Training Loss: 0.4597
Validation Loss: 0.5272

Epoch 3/5
Batch 0, Loss: 0.2401
Batch 10, L

In [ ]:
# prompt: the best model

from sklearn.metrics import accuracy_score, f1_score

# ... (Your existing code) ...

# After training loop:
# Load the best model
checkpoint = torch.load('best_model.pt')
model.load_state_dict(checkpoint['model_state_dict'])
liwc_layer.load_state_dict(checkpoint['liwc_layer_state_dict'])
classifier.load_state_dict(checkpoint['classifier_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

# Evaluate the best model on the test set
val_loss, predictions, true_labels = evaluate(model, test_loader, device)

# Calculate accuracy and F1-score
accuracy = accuracy_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions, average='weighted')

print(f'Best Model - Validation Loss: {val_loss:.4f}')
print(f'Accuracy: {accuracy:.4f}')
print(f'F1 Score: {f1:.4f}')


Best Model - Validation Loss: 0.5200
Accuracy: 0.7441
F1 Score: 0.7437


In [ ]:
# Cell: Test predictions with example texts
def predict_example(text, model, tokenizer, liwc_layer, classifier, device):
    # Preprocess text
    cleaned_text = clean_text(text)

    # Tokenize
    encoded = tokenizer(
        cleaned_text,
        padding='max_length',
        truncation=True,
        max_length=128,
        return_tensors='pt'
    ).to(device)

    # Create dummy LIWC features (zeros since we don't have LIWC features for new text)
    dummy_liwc = torch.zeros((1, 11)).to(device)

    # Get prediction
    model.eval()
    liwc_layer.eval()
    classifier.eval()
    with torch.no_grad():
        outputs = model(**encoded)
        bert_features = outputs.hidden_states[-1][:, 0, :]
        liwc_output = liwc_layer(dummy_liwc)
        combined_features = torch.cat((bert_features, liwc_output), dim=1)
        logits = classifier(combined_features)
        probabilities = torch.softmax(logits, dim=1)

    # Get prediction and confidence
    predicted_class = torch.argmax(probabilities, dim=1).item()
    confidence = probabilities[0][predicted_class].item()

    return {
        'text': cleaned_text,
        'stress_level': predicted_class,
        'confidence': confidence,
        'prediction': 'Stressed' if predicted_class == 1 else 'Not Stressed'
    }

# Test examples
test_examples = [
    "I'm feeling really overwhelmed with work and can't sleep properly. Everything is piling up and I can't handle it.",
    "Just had a great day at work! Everything went smoothly and I'm feeling productive.",
    "My anxiety is through the roof. I can't focus on anything and my heart is racing.",
    "Had a normal day today. Nothing special happened, just the usual routine.",
    "The deadline is approaching and I haven't even started. I'm panicking and don't know what to do.",
    "Spent a relaxing weekend with family. We watched movies and had good food.",
    "I've been having constant headaches and my mind won't stop racing. The pressure is unbearable.",
    "Meeting went well today. The team liked my presentation and the client was happy."
]

# Print predictions
print("Model Predictions:")
print("-" * 80)
for text in test_examples:
    result = predict_example(text, model, tokenizer, liwc_layer, classifier, device)
    print(f"\nText: {text}")
    print(f"Prediction: {result['prediction']}")
    print(f"Confidence: {result['confidence']:.2f}")
    print("-" * 80)

# Additional analysis
def analyze_keywords(text, prediction, confidence):
    """Analyze stress-related keywords in the text"""
    stress_keywords = ['anxiety', 'stress', 'overwhelm', 'panic', 'worry', 'pressure',
                      "can't", 'difficult', 'hard', 'tired', 'exhausted']
    positive_keywords = ['great', 'happy', 'relax', 'good', 'smooth', 'well', 'normal']

    found_stress = [word for word in stress_keywords if word in text.lower()]
    found_positive = [word for word in positive_keywords if word in text.lower()]

    return {
        'stress_indicators': found_stress,
        'positive_indicators': found_positive,
        'prediction': prediction,
        'confidence': confidence
    }

# Test with detailed analysis
print("\nDetailed Analysis:")
print("-" * 80)
for text in test_examples:
    result = predict_example(text, model, tokenizer, liwc_layer, classifier, device)
    analysis = analyze_keywords(text, result['prediction'], result['confidence'])

    print(f"\nText: {text}")
    print(f"Prediction: {result['prediction']}")
    print(f"Confidence: {result['confidence']:.2f}")
    print(f"Stress indicators found: {', '.join(analysis['stress_indicators']) if analysis['stress_indicators'] else 'None'}")
    print(f"Positive indicators found: {', '.join(analysis['positive_indicators']) if analysis['positive_indicators'] else 'None'}")
    print("-" * 80)

Model Predictions:
--------------------------------------------------------------------------------

Text: I'm feeling really overwhelmed with work and can't sleep properly. Everything is piling up and I can't handle it.
Prediction: Stressed
Confidence: 0.78
--------------------------------------------------------------------------------

Text: Just had a great day at work! Everything went smoothly and I'm feeling productive.
Prediction: Not Stressed
Confidence: 0.97
--------------------------------------------------------------------------------

Text: My anxiety is through the roof. I can't focus on anything and my heart is racing.
Prediction: Stressed
Confidence: 0.75
--------------------------------------------------------------------------------

Text: Had a normal day today. Nothing special happened, just the usual routine.
Prediction: Not Stressed
Confidence: 0.96
--------------------------------------------------------------------------------

Text: The deadline is approaching 

In [ ]:


from google.colab import files

# Save the model to your Google Drive
!cp best_model.pt /content/drive/MyDrive/

# Download the model file
files.download('best_model.pt')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>